# Overlapping records in two datasets

In [1]:
import numpy as np
import pandas as pd
import jellyfish
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)

# Read datasets to match

Read datasets A and B into Pandas data frames A and B, respectively.

In [38]:
df_a = pd.read_csv('datasetA.csv', dtype=str)
df_a

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
0,lachlan,carmody,19,cade place,b,hamilton,4556,qld,19960616,31.0,02 92812617,2231206
1,sarah,tippins,3,keverstone circuit,NaN,woodpark,5540,qld,19280526,13.0,04 11695119,4612010
2,jessica,cheshire,12,halfrey circuit,NaN,jesmond,5172,nsw,19210205,26.0,02 02242599,3144617
3,natalia,tiller,73,eagle circuit,NaN,camden,5114,nt,19351203,33.0,07 91923807,1245778
4,lily,blackwell,18,bindel street,cambooya,oak flats,6230,nsw,19791106,NaN,04 45361517,2195128
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,jessica,clarke,52,NaN,msf 60,chelmsford,2566,nsw,19601129,NaN,03 40798285,4849937
9996,alessandra,moody,514,withnell circuit,NaN,narrogin,6207,vic,19210819,NaN,03 44448442,3068581
9997,darcie,burford,4,longerenong street,cambridge park,ballarat,NaN,nsw,19970902,33.0,04 31228862,9011666
9998,steven,white,70,investigator street,NaN,parkdale,2640,nsw,19280106,34.0,04 90977051,1490648


In [39]:
df_b = pd.read_csv('datasetB.csv', dtype=str)
df_b

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
0,baysen,scudds,11,helemontstreet,NaN,picton,4055,vic,19340808,29.0,03 82765568,1070893
1,jsohua,greej,30,muecke place,rockview,willetton,6023,wa,19164909,NaN,02 30027583,5782396
2,alicia,treumekr,24,hambidge crescent,NaN,bligh park,4216,qld,19000819,21.0,08 11540291,7788784
3,anna,reid,13,creswellsstreet,oxonia,parksie,4005,vim,19807429,NaN,03 26039045,1646214
4,dato,natasha,96,de burgh street,NaN,bayswater,4670,vic,19301075,23.0,08 76809985,4308494
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ema,britten,4,mugga fay,NaN,north beach,2211,ws,19150617,42.0,02 42401723,9558954
9996,keegan,herbdrt,56,dunbar sxtreet,dutch care hostel,como,4065,qld,NaN,30.0,NaN,8854365
9997,makenzie,huxldey,6,frencham place,NaN,longrgeach,5271,wa,19810514,29.0,07 53146771,4011126
9998,sybella,irizsrry,5,mckinlay street,NaN,vermont,3141,qld,19890909,35.0,08 92355273,7620274


# Examples of matches

In [5]:
df_a.loc[[962, 2847, 7787, 6724]]

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
962,NaN,fleet,65,dartnell street,NaN,reynella,4215,nsw,19160720,23.0,02 46529999,1933859
2847,cameron,lademan,19,ebden street,NaN,williamstown,3165,nsw,19430515,29.0,08 39803141,8805927
7787,jamie,caruana,18,packham place,NaN,sunshine north,2096,nsw,19590202,23.0,08 71094666,5726002
6724,sophie,haupt,6,macrossan crescent,NaN,kyogle,2534,nsw,19071127,33.0,08 51432405,8085321


In [6]:
df_b.loc[[2704, 6268, 4843, 7373]]

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
2704,NaN,fleent,65,street datnell,NaN,reynella,4215,nsw,19160720,23.0,02 45629999,1933859
6268,camern,lademan,199,ebden street,NaN,bronte,3615,nsw,19430515,29.0,08 39803141,8805927
4843,jamie,caruana,18,packhamplace,NaN,sunshine north,2069,nzw,19590202,23.0,08 12512838,5726002
7373,sophie,haupt,6,macrossan yescent,NaN,kyogle,2534,nsw,NaN,33.0,08 51432405,4114985


# Exact matches

Merging the data frames by requiring one or more exact matches **does
not find all overlapping records**, since it does not take into account
differences between the values of the match columns in data frame A
and data frame B.

In [7]:
EXACT_COLS = ['last_name', 'date_of_birth', 'address_1']

In [8]:
df_a[EXACT_COLS].notnull().all(axis=1).sum()

8472

In [9]:
df_b[EXACT_COLS].notnull().all(axis=1).sum()

7824

Merge data frames A and B on specified columns for exact matching

In [10]:
exact_merge = df_a[df_a[EXACT_COLS].notnull().all(axis=1)]\
                  .reset_index(drop=False)\
                  .merge(df_b[df_b[EXACT_COLS].notnull().all(axis=1)].reset_index(drop=False),
                         how='inner', on=EXACT_COLS, suffixes=('_a', '_b'))\
                  .set_index(['index_a', 'index_b'] + EXACT_COLS, drop=True)
exact_merge = exact_merge[sorted(exact_merge.columns)]
exact_merge

,,,,,address_2_a,address_2_b,age_a,age_b,first_name_a,first_name_b,phone_number_a,phone_number_b,soc_sec_id_a,soc_sec_id_b,state_a,state_b,street_number_a,street_number_b,town_a,town_b,zipcode_a,zipcode_b
index_a,index_b,last_name,date_of_birth,address_1,,,,,,,,,,,,,,,,,,
4,7442,blackwell,19791106,bindel street,cambooya,cambooya,NaN,NaN,lily,liplt,04 45361517,04 45361517,2195128,2195128,nsw,nsw,18,1,oak flats,oak flats,6230,6230
11,1897,hyland,19770727,badimara street,NaN,lst hse on left,32.0,8.0,talissa,talsxa,07 50826244,07 50826244,8476381,8476381,qld,qld,1,1,carnegie,carnegie,5163,5163
16,2709,webb,19431014,chevalley loop,NaN,NaN,11.0,14.0,adam,ada,07 20680550,07 20680550,2826430,2826430,vic,vic,10,10,mont albert,mont albert,NaN,NaN
32,9344,badman,19960914,jarrahdale street,NaN,NaN,23.0,23.0,shannon,shannon,02 29205913,02 29205193,6953470,6953470,qld,qld,26,26,bonnells bay,bonnell bay,2257,2257
36,5272,wooley,19530721,trussell place,knackery rhs,knackery rhs,26.0,29.0,dante,dante,NaN,NaN,9656996,9656996,qld,qld,3,3,underdale,undedae,4858,4858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936,4026,matthews,19150613,ruthven street,caravn park,caravn park,21.0,21.0,jacobie,jacobue,08 77920512,08 77920512,3246160,3246160,qld,qkd,319,318,ryde,ryd,2567,2567
9962,2052,whale,19160324,deamer crescent,NaN,NaN,NaN,NaN,alana,alnam,07 99388950,07 99288950,1227704,1227604,vic,vic,96,96,yamba,yamba,4350,4350
9964,1900,manson,19330804,muresk street,NaN,NaN,33.0,33.0,noah,noah,02 65493761,02 65493761,9410525,9410525,vic,vci,6,8,elanora heights,elanoraheights,3555,3555


Print number of matches (incomplete set and possibly includes incorrect matches)

In [11]:
print('Exact merge on {0:s} gives {1:d} overlapping records'.format(str(EXACT_COLS), len(exact_merge)))

Exact merge on ['last_name', 'date_of_birth', 'address_1'] gives 905 overlapping records


In [ ]:
'''Start my code'''

In [73]:
import pandas as pd
from fuzzywuzzy import fuzz
from datasketch import MinHash, MinHashLSH

def preprocess_dataframe(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.lower() #convert to lowercase
            df[col] = df[col].str.strip() #rewove whitespace
            df[col] = df[col].fillna('') #convert missing values to empty string
    return df

df_a_processed = preprocess_dataframe(df_a.copy())  
df_b_processed = preprocess_dataframe(df_b.copy())

In [76]:
def missing_values(df):
    '''Determine percentage of non missing values per collumn'''
    total_num = 10000  
    for col in df.columns:
        non_missing_num = df[col].count()  
        non_missing_percentage = (non_missing_num / total_num) * 100
        print(f" '{col}' "f"{non_missing_percentage:.2f}")

missing_values(df_a)
missing_values(df_b)

def feature_extraction(row):
    '''Extract features from a rows with non_missing_percentage > 95'''
    features = {}
    features['first_name'] = row['first_name'] 
    features['last_name'] = row['last_name'] 
    features['street_number'] = row['street_number']
    features['address_1'] = row['address_1']
    features['town'] = row['town']
    features['zipcode'] = row['zipcode']
    features['soc_sec_id'] = row['soc_sec_id'] 
    return features

feature_list_a = df_a_processed.apply(feature_extraction, axis=1).tolist()
feature_list_b = df_b_processed.apply(feature_extraction, axis=1).tolist()

 'first_name' 98.09
 'last_name' 98.03
 'street_number' 97.77
 'address_1' 96.32
 'address_2' 39.62
 'town' 98.99
 'zipcode' 99.16
 'state' 89.70
 'date_of_birth' 89.74
 'age' 79.60
 'phone_number' 95.03
 'soc_sec_id' 100.00
 'first_name' 97.15
 'last_name' 96.68
 'street_number' 96.68
 'address_1' 94.42
 'address_2' 33.01
 'town' 98.57
 'zipcode' 98.87
 'state' 87.68
 'date_of_birth' 85.77
 'age' 76.93
 'phone_number' 93.84
 'soc_sec_id' 100.00


In [97]:
def jaccard_similarity(cand1, cand2):
    '''Calculate Jaccard similarity'''
    cand1_set = set(cand1)
    cand2_set = set(cand2)
    intersection = len(cand1_set.intersection(cand2_set))
    union = len(cand1_set.union(cand2_set))
    if union == 0:
        return 0  
    return intersection / union

#known match indices
match_a = [962, 2847, 7787, 6724]
match_b = [2704, 6268, 4843, 7373]

# Calculate Jaccard similarities
jaccard_sim = []
for indx in range(4):
    jaccard_sim.append(jaccard_similarity(df_a.loc[match_a[indx]],df_b.loc[match_b[indx]]))

print("Jaccard similarities:")
print(jaccard_sim)

Jaccard similarities:
[0.5714285714285714, 0.5, 0.5, 0.6428571428571429]


In [98]:
lsh = MinHashLSH(threshold=0.5, num_perm=256)
for i, feature_a in enumerate(feature_list_a):
    m = MinHash(num_perm=256)
    #update MinHash with all relevant features
    m.update(feature_a['first_name'].encode('utf8'))
    m.update(feature_a['last_name'].encode('utf8'))
    m.update(feature_a['street_number'].encode('utf8'))
    m.update(feature_a['address_1'].encode('utf8'))
    m.update(feature_a['town'].encode('utf8'))
    m.update(feature_a['zipcode'].encode('utf8'))
    m.update(feature_a['soc_sec_id'].encode('utf8')) 
    lsh.insert(i, m)

candidate_pairs = []
for j, feature_b in enumerate(feature_list_b):
    m = MinHash(num_perm=256)
    #update MinHash with all relevant features
    m.update(feature_b['first_name'].encode('utf8'))
    m.update(feature_b['last_name'].encode('utf8'))
    m.update(feature_b['street_number'].encode('utf8'))
    m.update(feature_b['address_1'].encode('utf8'))
    m.update(feature_b['town'].encode('utf8'))
    m.update(feature_b['zipcode'].encode('utf8'))
    m.update(feature_b['soc_sec_id'].encode('utf8'))

    #determine candidate pairs
    results = lsh.query(m)
    for result in results:
        candidate_pairs.append((result, j))

print("Number of candidate pairs:")
print(len(candidate_pairs))

nNumber of candidate pairs:
3306
